In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import imutils
import glob
import random
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from scipy.spatial import distance
from tensorflow.keras import datasets, layers, models
from skimage import io
import PIL

In [2]:
#model config paths
weights = glob.glob("yolov4-tiny-custom_final.weights")[0]
labels = glob.glob("labels.txt")[0]
cfg = glob.glob("yolov4-tiny-custom.cfg")[0]
print("You are now using {} weights ,{} configs and {} labels.".format(weights, cfg, labels))

You are now using yolov4-tiny-custom_final.weights weights ,yolov4-tiny-custom.cfg configs and labels.txt labels.


In [3]:
#load model
lbls = []
with open(labels, "r") as f:
    lbls = [c.strip() for c in f.readlines()]

COLORS = np.random.randint(0, 255, size=(len(lbls), 3), dtype="uint8")

net = cv2.dnn.readNetFromDarknet(cfg, weights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

layer = net.getLayerNames()
layer = [layer[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [4]:
CONFIDENCE_THRESHOLD= 0.5
NMS_THRESHOLD= 0.4

In [5]:
def detect(image, nn):
    #image = cv2.imread(imgpath)
    #image = cv2.imdecode(np.fromfile(imgpath, dtype=np.uint8), cv2.IMREAD_UNCHANGED)

    (H, W) = image.shape[:2]

    blob = cv2.dnn.blobFromImage(image, 1/255, (416, 416), swapRB=True, crop=False)
    nn.setInput(blob)
    start_time = time.time()
    layer_outs = nn.forward(layer)
    end_time = time.time()

    boxes = []
    confidences = []
    class_ids = []

    for output in layer_outs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > CONFIDENCE_THRESHOLD:
                box = detection[0:4] * np.array([W, H, W, H])
                (center_x, center_y, width, height) = box.astype("int")

                x = int(center_x - (width / 2))
                y = int(center_y - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
    bbcoordinates=[]
    classnum=[]
    
    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            color = [int(c) for c in COLORS[class_ids[i]]]
            cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
            text = "{}: {:.4f}".format(lbls[class_ids[i]], confidences[i])
#             cv2.putText(image, text, (x, y -5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            label = "Inference Time: {:.2f} ms".format(end_time - start_time)
#             cv2.putText(image, label, (0, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
            bbcoordinates.append((x,y,w,h))
            classnum.append(class_ids[i])
        #print(bbcoordinates)
#         print(class_ids[i])
        cv2.imshow("image", image)
        cv2.imwrite("output.png", image)
        cv2.waitKey(0)
        return image, bbcoordinates, classnum



In [6]:
IMG_WIDTH=416
IMG_HEIGHT=416

In [7]:
img=cv2.imread('test.jpg')
# img = np.asarray(bytearray(blob.download_as_bytes()), dtype=np.uint8)
# img = cv2.imdecode(img, flags=1)
img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))

In [8]:
image,bbox,id=detect(img,net)

In [9]:
def crop_objects(image, bbox, classname):
    counts=dict()
    for i in range(len(bbox)):
        ing_name= classname[i]
        counts[ing_name] = counts.get(ing_name, 0) + 1
        x,y,w,h= bbox[i]
        #print(x,y,w,h)
        abs_x=abs(x)
        abs_y=abs(y)
        abs_w=abs(w)
        abs_h=abs(h)
        print(abs_x, abs_y,abs_w, abs_h)
        cropped_img = image[abs_y: abs_y+abs_h, abs_x: abs_x+abs_w]
        # construct image name and join it to path for saving crop properly
        #img_name = ing_name + '_' + str(counts[ing_name]) + '.jpg'
        img_name= str(i)+'.jpg'
        
        if not os.path.isdir("cropped_images/demo_folder"):
            os.makedirs("cropped_images/demo_folder")
        
        path=r"cropped_images/demo_folder"
        img_path = os.path.join(path, img_name )
        # save image
        cv2.imwrite(img_path, cropped_img)

In [10]:
crop_objects(image,bbox, id)

88 190 102 50
107 108 93 54
284 278 91 49
193 56 54 38
317 232 87 43


In [11]:
def vconcat_resize_min(im_list, interpolation=cv2.INTER_CUBIC):
    w_min = min(im.shape[1] for im in im_list)
    im_list_resize = [cv2.resize(im, (w_min, int(im.shape[0] * w_min / im.shape[1])), interpolation=interpolation)
                      for im in im_list]
    return cv2.vconcat(im_list_resize)

In [12]:
def hconcat_resize_min(im_list, interpolation=cv2.INTER_CUBIC):
    h_min = min(im.shape[0] for im in im_list)
    im_list_resize = [cv2.resize(im, (int(im.shape[1] * h_min / im.shape[0]), h_min), interpolation=interpolation)
                      for im in im_list]
    return cv2.hconcat(im_list_resize)


In [13]:
def concat_tile_resize(img_list_2d, interpolation=cv2.INTER_CUBIC):
    img_list_v=[hconcat_resize_min(img_list_h, interpolation=cv2.INTER_CUBIC) for img_list_h in img_list_2d]
    return vconcat_resize_min(img_list_v, interpolation=cv2.INTER_CUBIC)


In [14]:
def preprocessing(img):
    img2=img
    img3=img
    height=img.shape[0]
    width=img.shape[1]
    if height>width and (width/height)<0.5:
        image = hconcat_resize_min([img, img2, img3])
    elif height<width and (height/width)<0.5:
        image = vconcat_resize_min([img, img2, img3])
    elif height<=100 and width<=100:
        image= concat_tile_resize([[img,img2,img3],[img,img2,img3],[img,img2,img3]])
    else:
        image=img
    return image

In [15]:
new_model = tf.keras.models.load_model('sia_model_new.h5', compile=False)
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 196, 196, 32)      4640      
                                                                 
 conv2d_2 (Conv2D)           (None, 196, 196, 64)      2112      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 98, 98, 64)       0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 96, 96, 16)        9232      
                                                                 
 conv2d_4 (Conv2D)           (None, 94, 94, 32)        4640      
                                                        

In [16]:
# path to the folder containing the images
folder_path = 'cropped_images/demo_folder'

# create an empty list to store the images
images = []

# loop through all the files in the folder
for filename in os.listdir(folder_path):
    # only read image files
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # read the image
        img = cv2.imread(os.path.join(folder_path, filename))
        # add the image to the list
        images.append(img)

In [17]:
w = 200
h = 200
columns = 3
rows = 1

In [18]:
duplicates_img_list=[]
duplicates_bbox_list=[]
duplicates_indices=[]
i=0

while i<len(images):
    anchor= images[i]
    anchor= preprocessing(anchor)
    anchor= cv2.resize(anchor,(w,h))
    anchor_bbox= bbox[i]
    
    for j in range(i+1,len(images)):
        positive= images[j]
        positive= preprocessing(positive)
        positive= cv2.resize(positive,(w,h))
        positive_bbox= bbox[j]
        negative= images[j]
        negative= preprocessing(negative)
        negative= cv2.resize(negative,(w,h))
        
        image_set= np.stack([anchor/ 255. , positive/ 255. , negative/ 255. ], axis=0)
        output = new_model.predict(image_set)
        
        post_dst= 0.5*(tf.math.reduce_variance(tf.subtract(output[0,:], output[1,:]))   / \
                    (tf.math.reduce_variance(output[0,:])+tf.math.reduce_variance(output[1,:])))
        print(post_dst.numpy())
        
        cv2.imshow('anc',anchor)
        cv2.imshow('pos',positive)
        cv2.waitKey(0)
        
        if post_dst <0.04:
            duplicates_img_list.append(positive)
            duplicates_bbox_list.append(positive_bbox)
            duplicates_indices.append((i,j))
            images.pop(j)
            bbox.pop(j)
            break
    i += 1
    


1/1 [==============================] - 2s 2s/step
0.048391122
1/1 [==============================] - 0s 26ms/step
0.078751296
1/1 [==============================] - 0s 26ms/step
0.025918338
1/1 [==============================] - 0s 31ms/step
0.06726191
1/1 [==============================] - 0s 26ms/step
0.017228803


In [19]:
print(len(images), len(bbox))
print(len(duplicates_img_list), len(duplicates_bbox_list))

3 3
2 2


In [20]:
for i in images:
    cv2.imshow('a',i)
    cv2.waitKey(0)
    

In [ ]:
for i in duplicates_img_list:
    cv2.imshow('d', i)
    cv2.waitKey(0)

In [21]:
bbox

[(88, 190, 102, 50), (107, 108, 93, 54), (284, 278, 91, 49)]

In [22]:
duplicates_bbox_list

[(193, 56, 54, 38), (317, 232, 87, 43)]

In [23]:
duplicates_indices

[(0, 3), (1, 3)]